# Agent class
Basically the actor-critic NN

In [1]:
# importing libraries
import numpy as np
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical

In [2]:
# init layer
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
  torch.nn.init.orthogonal_(layer.weight, std)
  torch.nn.init.constant_(layer.bias, bias_const)
  return layer


In [3]:
# agent class
class Agent(nn.Module):
  def __init__(self, envs):
      super(Agent, self).__init__()
      self.critic = nn.Sequential(
          layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, 1), std=1.0),
      )
      self.actor = nn.Sequential(
          layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, 64)),
          nn.Tanh(),
          layer_init(nn.Linear(64, envs.single_action_space.n), std=0.01),
      )

  def get_value(self, x):
      return self.critic(x)

  def get_action_and_value(self, x, action=None):
      logits = self.actor(x)
      probs = Categorical(logits=logits)
      if action is None:
          action = probs.sample()
      return action, probs.log_prob(action), probs.entropy(), self.critic(x)